In [1]:
import pandas as pd


In [26]:
file_path = './IRM-MarketData31Oct2019.xlsx'
volcube_data = pd.read_excel(file_path, sheet_name='VolCube_D')

In [27]:
volcube_data = pd.read_excel(file_path, sheet_name='VolCube_D')

In [28]:
# Impostiamo i nomi delle colonne utilizzando i valori dalla seconda riga.
volcube_data.columns = volcube_data.iloc[1]

# Rimuoviamo le prime due righe che contengono dati non necessari.
volcube_data = volcube_data.drop(volcube_data.index[:2])

# Reset dell'indice del DataFrame.
volcube_data.reset_index(drop=True, inplace=True)

# Rimuoviamo le prime due colonne che non contengono dati utili per la calibrazione.
volcube_data = volcube_data.iloc[:, 2:]

volcube_data.head()

1,NaN,NaN,Expiry,Tenor,-150.0,-100.0,-50.0,-25.0,0.0,25,50.0,100.0,150.0
0,1m2y,2000.083333,0.083333,2,NaN,NaN,0.00,2.50,12.1,2.92,0.00,0.00,0.0
1,1m5y,5000.083333,0.083333,5,NaN,0.00,0.00,1.48,18.0,1.82,4.53,0.00,0.0
2,1m10y,10000.083333,0.083333,10,0.00,0.00,13.65,4.72,29.9,-0.07,2.30,8.04,0.0
3,1m20y,20000.083333,0.083333,20,0.00,33.39,11.80,4.62,29.8,-1.3,-0.39,3.78,0.0
4,1m30y,30000.083333,0.083333,30,80.23,37.41,14.43,6.20,30.2,-3.24,-3.74,0.00,0.0


# Shifted SABR

$$
\sigma(F, K, T, \alpha, \beta, \rho, \nu, f) = \frac{\alpha \left(1 + \left( \frac{(1-\beta)^2 \alpha^2}{24 (F+f)^{1-\beta} (K+f)^{1-\beta}} + \frac{\rho \beta \nu \alpha}{4 (F+f)^{(1-\beta)/2} (K+f)^{(1-\beta)/2}} + \frac{(2-3\rho^2) \nu^2}{24} \right) T\right)}{(F+f)^{(1-\beta)/2} (K+f)^{(1-\beta)/2} \left( \frac{z}{\chi(z)} \right)}
$$

dove: <br>
$z = \frac{\nu}{\alpha} \left( \sqrt{F + f} \sqrt{K + f} \right)^{1-\beta} \ln \left( \frac{\sqrt{F + f}}{\sqrt{K + f}} \right)$     

e

$\chi(z) = \ln \left( \frac{\sqrt{1 - 2 \rho z + z^2} + z - \rho}{1 - \rho} \right)$


In [29]:
def shifted_sabr_volatility(F, K, T, alpha, beta, rho, nu, shift):
    """
    Calcola la volatilità SABR traslata.
    
    Parametri:
    F : float - Tasso forward
    K : float - Prezzo di esercizio (strike price)
    T : float - Tempo alla scadenza
    alpha : float - Parametro di volatilità SABR
    beta : float - Parametro di elasticità SABR
    rho : float - Correlazione tra il tasso forward e la volatilità
    nu : float - Volatilità della volatilità
    shift : float - Traslazione applicata al modello SABR
    
    Ritorna:
    sigma : float - Volatilità SABR traslata
    """
    # Trasformazione log-normale o normale in base a beta
    if beta == 1:  # Log-normale
        F += shift
        K += shift
        z = nu/alpha * np.log(F/K)
        x_z = np.log((np.sqrt(1 - 2*rho*z + z**2) + z - rho) / (1 - rho))
        A = alpha * (F*K)**((beta - 1)/2)
    else:  # Normale
        z = nu/alpha * (F - K)
        x_z = np.log((np.sqrt(1 - 2*rho*z + z**2) + z - rho) / (1 - rho))
        A = alpha

    # Calcolo della volatilità
    sigma = A * (z / x_z) * (1 + (1 - beta)**2/24 * (np.log(F/K))**2 / ((F*K)**((1 - beta)/2)) + (1 - beta)**4/1920 * (np.log(F/K))**4)
    return sigma
